# Imports

In [ ]:
import cyclops.query.process as qp
from cyclops.processors.column_names import ENCOUNTER_ID, LENGTH_OF_STAY_IN_ER
from cyclops.query import gemini
from cyclops.query.gemini import get_interface

# Query

In [ ]:
def get_cohort():
    """Get delirium cohort."""
    table = gemini.patient_encounters().query
    table = qp.DropNulls("del_present")(table)
    table = qp.ConditionEquals("del_present", 3, not_=True)(table)

    return get_interface(table).run()


def get_room_transfers_for_cohort(cohort):
    """Get room transfer info for cohort."""
    table = gemini.room_transfers().query
    table = qp.GroupByAggregate(
        ENCOUNTER_ID, {ENCOUNTER_ID: ("count", "room_transfers")}
    )(table)
    room_transfers = get_interface(table).run()
    cohort = cohort.merge(room_transfers, on=ENCOUNTER_ID, how="left")

    return cohort


def get_scu_transfers_for_cohort(cohort):
    """Get SCU transfer info for cohort."""
    table = gemini.get_table("ip_scu")
    table = qp.GroupByAggregate(
        ENCOUNTER_ID, {ENCOUNTER_ID: ("count", "icu_transfers")}
    )(table)
    ip_scu = get_interface(table).run()
    cohort = cohort.merge(ip_scu, on=ENCOUNTER_ID, how="left")
    cohort["icu_transfers"] = cohort["icu_transfers"].fillna(0)

    return cohort


def get_er_for_cohort(cohort):
    """Get ER info for cohort."""
    table = gemini.er_admin().query
    table = qp.FilterColumns(
        [
            ENCOUNTER_ID,
            "admit_via_ambulance",
            "triage_level",
            LENGTH_OF_STAY_IN_ER,
        ]
    )(table)
    er_info = get_interface(table).run()
    cohort = cohort.merge(er_info, on="encounter_id", how="left")

    return cohort


cohort = get_cohort()
cohort = get_room_transfers_for_cohort(cohort)
cohort = get_scu_transfers_for_cohort(cohort)
cohort = get_er_for_cohort(cohort)

In [ ]:
cohort